### catalogo de datos madrid

In [1]:
#parse dcat xml
import xml.etree.ElementTree as ET
import urllib.request

ns = {
    'foaf': "http://xmlns.com/foaf/0.1/",
    'skos': "http://datos.gob.es/kos/sector-publico/sector/",
    'dcat': "http://www.w3.org/ns/dcat#",
    'rdfs': "http://www.w3.org/2000/01/rdf-schema#",
    'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    'dct': "http://purl.org/dc/terms/",
    'dc': "http://purl.org/dc/elements/1.1/",
    'time': "http://www.w3.org/2006/time#",
    'xsd': "http://www.w3.org/2001/XMLSchema#",
    'j.0': "http://www.w3.org/1999/xhtml/vocab#",
    'j.1': "http://a9.com/-/spec/opensearch/1.1/",
    'api': "http://purl.org/linked-data/api/vocab#",
}

with urllib.request.urlopen('https://datos.madrid.es/egob/catalogo.rdf') as f:
    tree = ET.parse(f)
    root = tree.getroot()

In [2]:
from datetime import datetime
ds_list = [{'indentifier': dataset.find('dct:identifier', ns).text,
            'dataset_title': dataset.find('dct:title', ns).text
            , 'theme': dataset.find('dcat:theme', ns).attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
            , 'keywords': dataset.find('dcat:keyword', ns).text
            , 'issued':  datetime.strptime(dataset.find('dct:issued', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'modified': datetime.strptime(dataset.find('dct:modified', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'frequency': dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns).text if  dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns) is not None else None
            , 'distribution_title': distribution.find('dct:title', ns).text if distribution.find('dct:title', ns) is not None else None
            , 'mediaType': distribution.find('dcat:mediaType', ns).text
            , 'accessURL': distribution.find('dcat:accessURL', ns).text
            }
           for dataset
           in root.findall('./dcat:Catalog/dcat:dataset/dcat:Dataset', ns)
           for distribution
           in dataset.findall('dcat:distribution/dcat:Distribution', ns)

           ]

len(ds_list)

8873

In [3]:
for d in ds_list[0:5]:
    print(d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,sep = '; ')

Parque Tecnológico de Valdemingómez (PTV): caracterizaciones de residuos tasa municipal; 2024; text/csv; https://datos.madrid.es/egob/catalogo/300736-0-tasa-residuos.csv
Parque Tecnológico de Valdemingómez (PTV): caracterizaciones de residuos tasa municipal; 2024; application/vnd.ms-excel; https://datos.madrid.es/egob/catalogo/300736-1-tasa-residuos.xlsx
Evaluación de las Unidades Distritales de Colaboración (UDC); Evaluación del conocimiento de las UDC 2024; text/csv; https://datos.madrid.es/egob/catalogo/300729-0-evaluación-unidades-distritales.csv
Evaluación de las Unidades Distritales de Colaboración (UDC); Evaluación del conocimiento de las UDC 2024; application/vnd.ms-excel; https://datos.madrid.es/egob/catalogo/300729-1-evaluación-unidades-distritales.xlsx
Evaluación de las Unidades Distritales de Colaboración (UDC); Evaluación de las actuaciones de las UDC 2024; text/csv; https://datos.madrid.es/egob/catalogo/300729-2-evaluación-unidades-distritales.csv


In [4]:
import requests

x = requests.head('https://datos.madrid.es/egob/catalogo/300724-12621499-Seccionado-censal-mapas.xls', allow_redirects=True, verify=False)

print(x.status_code)
print(x.url)
print(x.headers)

/home/jcaubin/code/github/codespaces-jupyter_dam_01/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datos.madrid.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
https://sigma.madrid.es/hosted/rest/services/CARTOGRAFIA/LIMITES_ADMINISTRATIVOS/MapServer
{'Cache-Control': 'max-age=00:00:00', 'Content-Type': 'text/html; charset=utf-8', 'Content-Encoding': 'gzip', 'Vary': 'Origin', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': 'AGS_ROLES="419jqfa+uOZgYod4xPOQ8Q=="; Version=1; Max-Age=60; Expires=Fri, 03 Jan 2025 12:06:46 GMT; Path=/hosted/rest; Secure; HttpOnly, TS01134391=01e90f9b8b7d3585b66be8fdb736bb2514dd339dbecb6e7f9b943ec2eef482a87f22451bf3fc460cc9dba965b479f2b62bffa890ec; Path=/; , TS014e5201=01e90f9b8bb1554dbd19708dff53f04c94b88d7618cb6e7f9b943ec2eef482a87f22451bf3697cc181150f9bc297930dbe4a5db1f517c51e5bef35d466cc86604f4d098e69; path=/hosted/rest', 'Date': 'Fri, 03 Jan 2025 12:05:46 GMT', 'Transfer-Encoding': 'chunked'}


/home/jcaubin/code/github/codespaces-jupyter_dam_01/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sigma.madrid.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
import requests
requests.packages.urllib3.disable_warnings() 
import csv

with open('../data/catalogo_validacion2.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i, d in enumerate(ds_list[500:]):
        #print(d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,sep = '; ')
        try:
            x = requests.head(d['accessURL'], allow_redirects=True, verify=False)
            #print('\t', x.status_code)
            #print('\t', x.headers)
            #print('\t', x.headers['Date'], x.headers['Last-Modified'], x.headers['Content-Type'], x.headers['Content-Length'],sep = '; ' )
            print(i, d['dataset_title'], d['distribution_title'], x.status_code, x.headers['Content-Type'], x.headers['Content-Length'],  sep = '; ')
            csv_writer.writerow([d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,x.status_code,x.url, x.headers['Date'], x.headers['Last-Modified'], x.headers['Content-Type'], x.headers['Content-Length'], x.elapsed])
        except:
            print(i, d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,x.status_code, 'ERROR', x.elapsed, sep='; ') 
            csv_writer.writerow([d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,x.status_code,x.url,x.elapsed, 'ERROR']) 

    



0; Parque Tecnológico de Valdemingómez (PTV): caracterizaciones de residuos tasa municipal; 2024; 200; text/csv; 30875
1; Parque Tecnológico de Valdemingómez (PTV): caracterizaciones de residuos tasa municipal; 2024; 200; application/vnd.openxmlformats-officedocument.spreadsheetml.sheet; 0
2; Evaluación de las Unidades Distritales de Colaboración (UDC); Evaluación del conocimiento de las UDC 2024; 200; text/csv; 32168
3; Evaluación de las Unidades Distritales de Colaboración (UDC); Evaluación del conocimiento de las UDC 2024; 200; application/vnd.openxmlformats-officedocument.spreadsheetml.sheet; 0
4; Evaluación de las Unidades Distritales de Colaboración (UDC); Evaluación de las actuaciones de las UDC 2024; 200; text/csv; 14014
5; Evaluación de las Unidades Distritales de Colaboración (UDC); Evaluación de las actuaciones de las UDC 2024; 200; application/vnd.openxmlformats-officedocument.spreadsheetml.sheet; 34566
6; Encuesta de uso del Portal de Datos Abiertos del Ayuntamiento de Mad